In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = Dense(128, activation='relu')(x)  
predictions = Dense(5, activation='softmax')(x)  
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train_datagen = ImageDataGenerator(
    rescale=1./255,       
    rotation_range=30,    
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


train_generator = train_datagen.flow_from_directory(
    'Dataset',  
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  
)


model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10
)


model.save('pomegranate_disease_classifier.h5')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Found 1350 images belonging to 5 classes.


d:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 135s 3s/step - accuracy: 0.6666 - loss: 0.8930
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 740us/step - accuracy: 0.9375 - loss: 0.1418
Epoch 3/10


d:\anaconda\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


42/42 ━━━━━━━━━━━━━━━━━━━━ 121s 3s/step - accuracy: 0.9362 - loss: 0.1966
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 246us/step - accuracy: 0.9688 - loss: 0.0731
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 120s 3s/step - accuracy: 0.9461 - loss: 0.1748
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 207us/step - accuracy: 0.9062 - loss: 0.1268
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 119s 3s/step - accuracy: 0.9612 - loss: 0.1040
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 320us/step - accuracy: 0.9688 - loss: 0.0526
Epoch 9/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 122s 3s/step - accuracy: 0.9587 - loss: 0.1332
Epoch 10/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 206us/step - accuracy: 0.9688 - loss: 0.0882


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os


model = tf.keras.models.load_model('pomegranate_disease_classifier.h5')
class_labels = ['Alternaria', 'Anthracnose','Bacterial_Blight', 'Cercospora','Healthy' ]

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224)) 
    img_array = image.img_to_array(img) 
    img_array = np.expand_dims(img_array, axis=0)  
    img_array = img_array / 255.0  
    return img_array


def predict_disease(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)  
    predicted_label = class_labels[predicted_class[0]]
    print(f"Predicted class: {predicted_label}")


img_path = 'Dataset/Anthracnose/IMG_20230813_151540_1.jpg'  # Path to the image you want to predict
predict_disease(img_path)


1/1 [==============================] - 1s 633ms/step
Predicted class: Anthracnose
